# Parameter Tunning Extra tree classifier

In [34]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from imblearn.over_sampling import SMOTE


import xgboost as xgb

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [41]:
labels_seba = pd.read_csv('../../data/cuarenta_cols.csv')

In [45]:
labels = pd.read_csv('../../data/labels_training_set.csv')

In [46]:
labels_seba = pd.merge(labels, labels_seba, on = 'person', how = 'inner')

In [49]:
labels_seba.drop('person',axis = 1)

,label,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,searched products mes 5,viewed product mes 5,dias_hasta_ultimo,viewed product mes 4,visited site mes 4,distan_dias,cant_dias_dist,modelos_dist,promedio_por_dia_x,promedio_por_mes
0,0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,...,0.0,23.0,0,0.0,0.0,8,10,12,6.800000,68.000000
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,0.0,0.0,0,1,0,2.000000,2.000000
2,0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,31.0,...,2.0,14.0,2,8.0,11.0,108,12,7,8.000000,24.000000
3,0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,24.0,...,0.0,24.0,4,0.0,0.0,8,4,11,9.250000,37.000000
4,0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,9.0,12,0.0,0.0,0,2,1,8.500000,17.000000
5,1,31.0,3.0,1.0,166.0,0.0,92.0,13.0,20.0,121.0,...,0.0,0.0,29,41.0,70.0,82,31,11,18.193548,141.000000
6,0,1.0,4.0,1.0,0.0,0.0,1.0,2.0,2.0,8.0,...,2.0,8.0,0,0.0,0.0,5,3,2,8.000000,24.000000
7,0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,54,0.0,3.0,1,3,0,2.666667,8.000000
8,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,4.0,9,0.0,0.0,0,1,1,11.000000,11.000000
9,0,2.0,1.0,0.0,2.0,0.0,2.0,0.0,0.0,12.0,...,0.0,12.0,13,0.0,0.0,0,1,1,22.000000,22.000000


In [11]:
labels_f = pd.read_csv('../../data/labels_f_filtrado.csv')
datos = pd.read_csv('../../data/datos_filtrado.csv')

In [12]:
atributos = labels_f.columns.tolist()

In [13]:
atributos.remove('person')
atributos.remove('label')

In [14]:
y = labels_f['label'].ravel()
X = labels_f.loc[:,atributos]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [16]:
def aplicarRus(X_train_p, y_train_p):
    rus= RandomUnderSampler(return_indices=True)
    #id_rus son los índices
    X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train_p, y_train_p)
    return(X_rus_train, y_rus_train)

In [17]:
X_train, y_train = aplicarRus(X_train,y_train)

In [18]:
modelo = ExtraTreesClassifier(n_estimators = 500,max_features=  0.5,max_depth= 8)

modelo.fit(X_train, y_train)
    
preds = modelo.predict(X_test)
preds_prob = modelo.predict_proba(X_test)[:,1]
train_accuracy = accuracy_score(y_train, modelo.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
matriz_de_confusion = confusion_matrix(y_test, preds)
print('train acurracy: ')
print(train_accuracy)
print('test acurracy: ')
print(test_accuracy)
print('Matriz de confusión: ')
print(matriz_de_confusion)
print('Área bajo la curva: ')
print(area_debajo_de_curva)
print()

train acurracy: 
0.9057562076749436
test acurracy: 
0.7631307929969104
Matriz de confusión: 
[[1406  442]
 [  18   76]]
Área bajo la curva: 
0.8505514875195728



In [29]:
model = ExtraTreesClassifier(n_estimators=100, n_jobs=4, min_samples_split=25,
                            min_samples_leaf=35)
                            
clf = GridSearchCV(
    estimator=model,
    param_grid={
        'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1600, num = 10)],
        #'max_features': range(50,401,50),
        'min_samples_leaf': [2, 5, 10],
        'min_samples_split': [2, 5, 10],
        'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)]
    },
    scoring='roc_auc',
)

clf.fit(X_train, y_train)

print(clf.best_score_)
print(clf.best_estimator_)

preds = clf.predict(X_test)
preds_prob = clf.predict_proba(X_test)[:,1]
train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
matriz_de_confusion = confusion_matrix(y_test, preds)
print('train acurracy: ')
print(train_accuracy)
print('test acurracy: ')
print(test_accuracy)
print('Matriz de confusión: ')
print(matriz_de_confusion)
print('Área bajo la curva: ')
print(area_debajo_de_curva)
print()


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while

0.8528741615127772
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=100, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
train acurracy: 
0.8589164785553047
test acurracy: 
0.7476828012358393
Matriz de confusión: 
[[1377  471]
 [  19   75]]
Área bajo la curva: 
0.8519273279911578



In [50]:
atributos = labels_seba.columns.tolist()

In [51]:
atributos.remove('person')
atributos.remove('label')

In [52]:
y = labels_seba['label'].ravel()
X = labels_seba.loc[:,atributos]

In [53]:
def aplicarSmote(X_train_p, y_train_p, X_test):
    smote = SMOTE(ratio='minority')
    X_smt_train, y_smt_train = smote.fit_sample(X_train_p, y_train_p)
    #X_smt_train = pd.DataFrame(X_smt_train)
    #X_smt_train.columns = X_test.columns
    return (X_smt_train, y_smt_train)

In [54]:
def aplicarRus(X_train_p, y_train_p,X_test_p):
    rus= RandomUnderSampler(return_indices=True)
    #id_rus son los índices
    X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train_p, y_train_p)
    #X_rus_train = pd.DataFrame(X_rus_train)
    #X_rus_train.columns = X_test_p.columns
    return(X_rus_train, y_rus_train)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [56]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)

In [57]:
def aplicarModelo(modelo,X_train_p, X_test_p, y_train_p, y_test_p, early = False):
    if(early):
        modelo.fit(X_train_p, y_train_p, early_stopping_rounds = 5, eval_set=[(X_test_p, y_test_p)])
    else: 
        modelo.fit(X_train_p, y_train_p)
    
    preds = modelo.predict(X_test_p)
    preds_prob = modelo.predict_proba(X_test_p)[:,1]
    train_accuracy = accuracy_score(y_train_p, modelo.predict(X_train_p))
    test_accuracy = accuracy_score(y_test_p, preds)
    area_debajo_de_curva = roc_auc_score(y_test_p, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test_p, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)
    print()

In [58]:
modelo = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=100, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
print('APLICO SMOTE')
aplicarModelo(modelo,X_smt_train, X_test, y_smt_train, y_test, False)
print('APLICO RUS')
aplicarModelo(modelo,X_rus_train, X_test, y_rus_train, y_test, False)

APLICO SMOTE
train acurracy: 
0.9860424454359098
test acurracy: 
0.9325437693099897
Matriz de confusión: 
[[1782   66]
 [  65   29]]
Área bajo la curva: 
0.8567686745878235

APLICO RUS
train acurracy: 
0.8656884875846501
test acurracy: 
0.7265705458290422
Matriz de confusión: 
[[1338  510]
 [  21   73]]
Área bajo la curva: 
0.8371039421571337

